# eCallisto Validation

The aim of this packages is to:

* Appending the MetaData into the datebase (<font color='red'> Save_To_Sql </font> ).
* Calculate the standard deviation and update it into the datebase (<font color='red'> STD </font>).
* Choosing 10 Spectrograms from each Station and plot them with 4 columns, then create a pdf file (<font color='red'> Testing_10000 </font>).


### Importing the Modules 

In [ ]:
import os, sys
import io
import glob
import astropy.io.fits
import matplotlib
import numpy as np
import time
import psycopg2
import pandas as pd
import pandas.io.sql as psql
from sqlalchemy import create_engine
import matplotlib.pyplot as plt

from eCallistoProject import plot_config

module_path = os.path.abspath(os.path.join('radiospectra'))
if module_path not in sys.path:
    sys.path.append(module_path)


import radiospectra
from radiospectra.sources import CallistoSpectrogram
from matplotlib.backends.backend_pdf import PdfPages
import warnings
warnings.filterwarnings("ignore")

* The path to the data in my notebook

In [ ]:
path = 'R:\\radio\\2002-20yy_Callisto\\2017\\09\\06'

## Save_To_Sql

* Converting the time (by Kushtrim).

In [ ]:
def __to_timestamp(date_string, time_string):
    
                sixty_seconds = int(time_string[6:8]) == 60
                sixty_minutes = int(time_string[3:5]) == 60
                twentyfour_hours = int(time_string[:2]) == 24
                
                # replacing  24 to 00 
                if sixty_seconds :
                    time_string = time_string[:6] + '59' + time_string[8:]
                if sixty_minutes :
                    time_string = time_string[:3] + '59' + time_string[5:]
                if twentyfour_hours :
                    time_string = '23' + time_string[2:]
                if re.findall("\.\d+", time_string):
                    time_string = time_string[:-4]
                    
                # lost time     
                ts = datetime.datetime.strptime(
                    '%s %s' % (date_string, time_string), '%Y/%m/%d %H:%M:%S')                  
                ts += datetime.timedelta(hours = int(twentyfour_hours),
                                         minutes = int(sixty_minutes),
                                         seconds = int(sixty_seconds))
                
                return ts  

* Establish a DBAPI connection at localhost:5432 when a connection request


In [ ]:
engine = create_engine("postgresql+psycopg2://" + 'postgres' + ":" + 'ecallistohackorange' + "@" + 'localhost' + "/" + 'validation')


### In this Script:
* Walk through the path to find the data
* Open the fits-file
* Call the Mata Data from the header list

In [ ]:
for root, dirs, files in os.walk(path):
    for file in files:
        if file.endswith('.fit.gz'):
            if file is not data_frame:
                
                instruments = os.path.join(root, file)

                hdulist = astropy.io.fits.open(instruments)
                # calling the mathaDaten from the Header list
                instrument_name = hdulist[0].header['INSTRUME'] 
                date_obs = hdulist[0].header['DATE-OBS'] 
                time_obs = hdulist[0].header['TIME-OBS']
                date_end = hdulist[0].header['DATE-END'] 
                time_end = hdulist[0].header['TIME-END'] 

* Combine date and time obs, date and time end.


In [ ]:
start_time = __to_timestamp(date_obs, time_obs)
end_time = __to_timestamp(date_end, time_end)

### In this Script:
* Creating a datafrane in pandas
* Inserting the data into the Dataframe

In [ ]:
# creating dataframe in pandas
data={
    'file_name':[file],
      'instrument_name':[instrument_name],
      'start_time':[start_time] ,
      'end_time':[end_time],
      'std' : [None]
      }
# Insert the data into the DataFrame
data_frame = pd.DataFrame(data, index = [df]) 

* Create a connection between pandas and sql.

In [ ]:
data_frame.to_sql('testing', con = engine, if_exists = 'append', chunksize = 50000, index=False)

## STD (Standard Deviation)

* Connect to the database Postgres

In [ ]:
connection = psycopg2.connect(user="postgres",
                              password="ecallistohackorange",
                              host="localhost",
                              port="5432",
                              database="validation")

cursor = connection.cursor()

* Select from the table "ecallisto" from Database to calculate the Std.

In [ ]:
cursor.execute("""SELECT * from  ecallisto WHERE std is null ORDER BY id""")

* Subtracted the background.
* Calculate the std.
* Update the std into the Database ecallisto.
* Exception to catch the all errors.
* Finally to close the connection.

In [ ]:
for file in cursor.fetchall():    
    full_path = os.path.join(path, file[1]) # 1 is the index of file_name in the cursor 
    try:
        spec = CallistoSpectrogram.read(full_path)

        # subtracted the background and then calculate the std.
        spec2 = spec.subtract_bg("subtract_bg_sliding_window", window_width=800, affected_width=1,
                                 amount=0.05, change_points=True)
        spec_std = spec2.data.std()
        
        # update the std into the Database ecallisto
        sql_update_query = f""" UPDATE ecallisto SET std = {spec_std} where id = {file[0]} """ 
        cursor.execute(sql_update_query)
        connection.commit()
        
        # expetion to catch the all errors.
    except Exception as err:
        
        exception_type = type(err).__name__
        print(exception_type, file[1]) 
        
        # close the connection
    finally:
        connection.close()
        print("Table after Updating the records")

## Testing_10'000

* Selecting 10 Spectogram for each instrument. 

In [ ]:
cursor.execute("""select * from (
                                 select ROW_NUMBER() OVER (partition by instrument_name order by id)
                                 as row_num, ecallisto.* FROM ecallisto
                                 ) t
                                 where row_num <=10
                                 order by instrument_name""")

To move the Axes:
* Source: https://gist.github.com/salotz/8b4542d7fe9ea3e2eacc1a2eef2532c5 
* By Kushtrim.

In [ ]:
def move_axes(fig, ax_source, ax_target):
    old_fig = ax_source.figure
    ax_source.remove()
    ax_source.figure = fig
    ax_source.set_ylabel('')
    ax_source.set_xlabel('')
        
    ax_source.set_position(ax_target.get_position())
    ax_target.remove()
    ax_target.set_aspect("equal")
    fig.axes.append(ax_source)
    fig.add_subplot(ax_source)
    
    plt.close(old_fig)

* To generate the Colormap (by Kushtrim)

In [ ]:
my_colormap = matplotlib.colors.LinearSegmentedColormap.from_list("myColorMap", plot_config.COLORMAP / 255)

* Create a pdf file

In [ ]:
with PdfPages('C:\\Users\\delbe\\OneDrive\\Desktop\\eCallisto_validation\\Plot_PDF.pdf') as pdf:

* The first column include the original data (Spectrogram).

In [ ]:
full_path = os.path.join(path, file[2]) # 2 is the index of file_name in the cursor
spec = CallistoSpectrogram.read(full_path)
fig1, axs1 = plt.subplots(1, 4, figsize=(25,5))
ax1 = spec.plot(cmap=my_colormap, colorbar=None)
ax1.title.set_text("Original Data")
plt.close()

* The second column include the 'Constbacksub + elimwrongchannels' by Kushtrim.

In [ ]:
spec2 = spec.subtract_bg("constbacksub", "elimwrongchannels")
fig2 = plt.subplots(1, 4, figsize=(25,5))
ax2 = spec2.plot(cmap=my_colormap, colorbar=None, vmin=-5, vmax=5)
ax2.title.set_text("Bg_cbs_rfi")
plt.close()

* The third Column include the 'subtract_bg_sliding_window' by Simon.

In [ ]:
spec3 = spec.subtract_bg("subtract_bg_sliding_window", window_width=800, affected_width=1,
                         amount=0.05, change_points=True)
fig3 = plt.figure(figsize=(25,5))
ax3 = spec3.plot(cmap=my_colormap, colorbar=None, vmin=-5, vmax=5)
ax3.title.set_text("Bg_sub_sliding_rfi")
plt.close()

* The fourth column include the Histograms for both.

In [ ]:
# the histogram for Constbacksub + elimwrongchannels
data_hist3 = np.absolute(spec2.data.flatten())

# the histogram for subtract_bg_sliding_window
data_hist4 = np.absolute(spec3.data.flatten())

fig4, ax4 = plt.subplots(figsize=(25,5))
ax4.hist(data_hist3 ,histtype='step',range= (0, 10), bins= 40, label='Bg_cbs_rfi')
ax4.hist(data_hist4 ,histtype='step',range= (0, 10), bins= 40, label='Bg_sub_sliding_rfi')
ax4.title.set_text("Histograms")
plt.legend()
plt.close()

* Plot final plot by moving axes to the figure.

In [ ]:
fig_target, (axA, axB, axC, axD) = plt.subplots(1, 4, figsize=(30,5))
plt.suptitle(fig1._suptitle.get_text())

* Move the axes by Kushtrim.

In [ ]:
move_axes(fig_target, ax1, axA)
move_axes(fig_target, ax2, axB)
move_axes(fig_target, ax3, axC)
move_axes(fig_target, ax4, axD)
plt.show()